In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
db_name = "google_repos_all.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS repositories")
cursor.execute("""
    CREATE TABLE repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        language TEXT,
        stars INTEGER
    )
""")
conn.commit()
base_url = "https://github.com/orgs/google/repositories"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
MAX_PAGES = 5000#最大ページ数
total_saved = 0
KNOWN_LANGUAGES = ["Python", "Java", "C++", "C", "Go", "JavaScript", "TypeScript", "HTML", "Dart", "Rust", "Shell", "Kotlin", "Swift", "Jupyter Notebook"]
print(f"スクレイピングを開始します: {base_url}")
print(f"対象のページ数は: {MAX_PAGES} ")
for page_num in range(1, MAX_PAGES + 1):
    target_url = f"{base_url}?page={page_num}"
    print(f"処理中[{page_num:<3}] ... ", end="", flush=True)
    try:
        response = requests.get(target_url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Error: Status {response.status_code}")
            break
        soup = BeautifulSoup(response.text, "html.parser")#ページ内のすべてのliタグを取得
        all_lis = soup.find_all("li")
        count_in_page = 0
        for li in all_lis: #リポジトリ名の抽出
            h3 = li.find("h3")#リポジトリ名が含まれるh3タグの取得
            if not h3: continue
            link = h3.find("a")#リポジトリリンクの取得
            if not link: continue
            href = link.get("href")#リンクのhref属性の取得
            if not href or "google" not in href: continue #Googleのリポジトリリンクか確認
            repo_name = link.get_text(strip=True)#リポジトリ名の取得
            language = "Unknown"

            #プログラミング言語の抽出
            lang_tag = li.find("span", itemprop="programmingLanguage")
            if lang_tag:
                language = lang_tag.get_text(strip=True)

            #カラードットの親要素
            if language == "Unknown":
                color_dot = li.find("span", class_=lambda c: c and "repo-language-color" in c)
                if color_dot and color_dot.parent:
                    text = color_dot.parent.get_text(strip=True)
                    for lang in KNOWN_LANGUAGES:
                        if lang in text:
                            language = lang
                            break
                    if language == "Unknown" and len(text) < 20:
                         language = text.replace("●", "").strip()
            #テキスト全体から探索
            if language == "Unknown":
                full_text = li.get_text()#liタグ内の全テキスト
                for lang in KNOWN_LANGUAGES:#既知の言語リストから探索
                    if lang in full_text:
                        language = lang
                        break
            #スター数の抽出
            stars = 0
            star_link = li.find("a", href=lambda h: h and h.endswith("/stargazers"))
            if star_link:
                raw_star = star_link.get_text(strip=True).replace(",", "")
                try:
                    if "k" in raw_star:
                         stars = int(float(raw_star.replace("k", "")) * 1000)
                    else:
                         stars = int(raw_star)
                except:
                    stars = 0
            #ファイルにデータを保存
            cursor.execute("INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)",
                           (repo_name, language, stars))
            count_in_page += 1
        conn.commit()
        if count_in_page > 0:#ページ内にリポジトリが存在した場合
            print(f"完了. ({count_in_page} リポジトリ)")#保存したリポジトリ数を表示
            total_saved += count_in_page#合計保存数に加算
        else:
            print("ページ内にリポジトリが存在しません。 (リストを終了)")#ページ内にリポジトリが存在しない場合、終了
            break
        time.sleep(1)
    except Exception as e:
        print(f"\nError: {e}")
        break
print("-" * 50)
print(f"スクレイピングが完了しました。 保存されたリポジトリの合計は: {total_saved}")
print("\n--- トップ30のスター数の多いリポジトリ ---")#トップ30のスター数の多いリポジトリを表示
cursor.execute("SELECT name, language, stars FROM repositories ORDER BY stars DESC LIMIT 30")
rows = cursor.fetchall()
print(f"{"順位":<5} | {"リポジトリ名":<35} | {"プログラミング言語":<15} | {"スター数":<10}")
print("-" * 75)
for i, row in enumerate(rows, 1):
    print(f"{i:<5} | {row[0]:<35} | {row[1]:<15} | {row[2]:<10}")
conn.close()

スクレイピングを開始します: https://github.com/orgs/google/repositories
対象のページ数は: 5000 
処理中[1  ] ... 完了. (30 リポジトリ)
処理中[2  ] ... 完了. (30 リポジトリ)
処理中[3  ] ... 完了. (30 リポジトリ)
処理中[4  ] ... 完了. (30 リポジトリ)
処理中[5  ] ... 完了. (30 リポジトリ)
処理中[6  ] ... 完了. (30 リポジトリ)
処理中[7  ] ... 完了. (30 リポジトリ)
処理中[8  ] ... 完了. (30 リポジトリ)
処理中[9  ] ... 完了. (30 リポジトリ)
処理中[10 ] ... 完了. (30 リポジトリ)
処理中[11 ] ... 完了. (30 リポジトリ)
処理中[12 ] ... 完了. (30 リポジトリ)
処理中[13 ] ... 完了. (30 リポジトリ)
処理中[14 ] ... 完了. (30 リポジトリ)
処理中[15 ] ... 完了. (30 リポジトリ)
処理中[16 ] ... 完了. (30 リポジトリ)
処理中[17 ] ... 完了. (30 リポジトリ)
処理中[18 ] ... 完了. (30 リポジトリ)
処理中[19 ] ... 完了. (30 リポジトリ)
処理中[20 ] ... 完了. (30 リポジトリ)
処理中[21 ] ... 完了. (30 リポジトリ)
処理中[22 ] ... 完了. (30 リポジトリ)
処理中[23 ] ... 完了. (30 リポジトリ)
処理中[24 ] ... 完了. (30 リポジトリ)
処理中[25 ] ... 完了. (30 リポジトリ)
処理中[26 ] ... 完了. (30 リポジトリ)
処理中[27 ] ... 完了. (30 リポジトリ)
処理中[28 ] ... 完了. (30 リポジトリ)
処理中[29 ] ... 完了. (30 リポジトリ)
処理中[30 ] ... 完了. (30 リポジトリ)
処理中[31 ] ... 完了. (30 リポジトリ)
処理中[32 ] ... 完了. (30 リポジトリ)
処理中[33 ] ... 完了. (30 リポジトリ)
処